In [7]:
import os
os.environ['KERAS_BACKEND'] = "torch"
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
%load_ext autoreload
%autoreload 2

In [22]:
from modules import encoding

## Retrieve data


In [99]:
df_match = pd.read_csv("../data/SAbDab/data.csv", sep=";")
# df_match[["ab_id", "ab_type"]] = df_match["ab"].str.split('|', n=1, expand=True)
# df_match[["ag_id", "ag_type"]] = df_match["ag"].str.split('|', n=1, expand=True)
df_match.head()

,ab,ag,interaction
0,5kel|ab,5kel|ag,1
1,5kel|ab,6cwt|ag,0
2,5kel|ab,4fp8|ag,0
3,5kel|ab,4yjz|ag,0
4,5kel|ab,6j15|ag,0


In [98]:
df_seq = pd.read_csv("../data/SAbDab/sequences.csv", sep=";")
df_seq.head()

,seq_id,specie,sequence
0,5kel|ag,Zaire ebolavirus (strain Mayinga-76) (128952),IPLGVIHNSTLQVSDVDKLVCRDKLSSTNQLRSVGLNLEGNGVATD...
1,5kel|ag,Zaire ebolavirus (128952),EAIVNAQPKCNPNLHYWTTQDEGAAIGLAWIPYFGPAAEGIYTEGL...
2,5kel|ab,Homo sapiens (9606),EVQLQESGGGLMQPGGSMKLSCVASGFTFSNYWMNWVRQSPEKGLE...
3,5kel|ab,Homo sapiens (9606),DIQMTQSPASLSVSVGETVSITCRASENIYSSLAWYQQKQGKSPQL...
4,5kel|ab,Homo sapiens (9606),DVKLLESGGGLVQPGGSLKLSCAASGFSLSTSGVGVGWFRQPSGKG...


In [83]:
seq = df_seq["sequence"]
encoder = encoding.alphabet_one_hot(alphabet=encoding.AMINO_ACID_ALPHABET)

In [84]:
seq_encode = encoding.one_hot_encoder(seq, encoder, vector_size=1500)

In [93]:
encoded = encoding.one_hot_encode_sequence("MYA", encoder, 100)

In [106]:
seq_encoded = encoding.one_hot_encoder(seq, encoder, 1500)

## Network